In [10]:
import cudf
import pandas as pd
%cd /data/dhk/physionet.org/files/mimiciv/2.2/hosp

/data/dhk/physionet.org/files/mimiciv/2.2/hosp


In [11]:
df_dh = cudf.read_csv("process_dohun.csv")

df_dh = df_dh.drop('subject_id', axis=1)
df_dh = df_dh.drop('hadm_id', axis=1)
df_dh

y = df_dh['hospital_expire_flag'] 
X = df_dh.drop('hospital_expire_flag', axis=1) # 'label' 열을 제외한 나머지 열을 X로 사용


In [12]:
import lightgbm as lgb
from cuml.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [13]:
model = lgb.LGBMClassifier()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state=42)

X_train = X_train.to_cupy()
X_test = X_test.to_cupy()
y_train = y_train.to_cupy()
y_test = y_test.to_cupy()

# X_train = X_train.to_numpy()
# X_test = X_test.to_numpy()
# y_train = y_train.to_numpy()
# y_test = y_test.to_numpy()

X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')


 # 매개변수 그리드 정의
param = {
  'num_leaves': [20, 30, 40],
  'learning_rate': [0.1, 0.01, 0.001],
  'n_estimators': [100, 500, 1000],
}

# GridSearchCV 객체 생성
grid_search = GridSearchCV(model, param_grid=param, cv=5, refit=True, n_jobs=-1, return_train_score=True, verbose=2, error_score='raise')
# 훈련 데이터로 모델 학습 및 최적 매개변수 탐색
best_model = grid_search.fit(X_train.get(), y_train.get())
# 최적 매개변수 출력

y_pred = best_model.predict(X_test)
     

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[LightGBM] [Info] Number of positive: 3311390, number of negative: 3312799
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 3.099931 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 711
[LightGBM] [Info] Number of data points in the train set: 6624189, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499894 -> initscore=-0.000425
[LightGBM] [Info] Start training from score -0.000425
[CV] END .learning_rate=0.1, n_estimators=100, num_leaves=20; total time=36.2min
[LightGBM] [Info] Number of positive: 3311389, number of negative: 3312800
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.019612 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 715
[LightGBM] [Info] Number 

/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Number of positive: 3311390, number of negative: 3312799
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.087796 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 711
[LightGBM] [Info] Number of data points in the train set: 6624189, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499894 -> initscore=-0.000425
[LightGBM] [Info] Start training from score -0.000425
[CV] END learning_rate=0.1, n_estimators=500, num_leaves=40; total time=251.0min
[LightGBM] [Info] Number of positive: 3311389, number of negative: 3312800
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.952359 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 715
[LightGBM] [Info] Numbe

TypeError: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.

In [14]:
y_pred = best_model.predict(X_test.get())

In [15]:
import cuml
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

cu_score = cuml.metrics.accuracy_score( y_test, y_pred )
print(cu_score)

# 정확도 계산
accuracy = accuracy_score(y_test.get(), y_pred)
print("Accuracy:", accuracy)

# F1 스코어 계산
f1 = f1_score(y_test.get(), y_pred)
print("F1 Score:", f1)

0.9559462070465088
Accuracy: 0.9559462044578418
F1 Score: 0.9568415801942824


In [16]:
import pandas as pd

# randomForest_y_pred를 Pandas DataFrame으로 변환
y_pred_df = pd.DataFrame(y_pred)

# CSV 파일로 저장
y_pred_df.to_csv('lightgbm_y_pred.csv', index=False, header=False)

In [2]:
# import torch

# torch.cuda.empty_cache()